In [1]:
# May need to install these....
# uncomment if unsure
#!pip install wget
#!pip install dtale

In [2]:
import pandas as pd
import numpy as np
import os

import dtale # data Frame visualization
import wget

# Download the data

Download Current employment statistics from [US Bureau of Labor Statics website](https://www.bls.gov/)


Data Dictionary
https://download.bls.gov/pub/time.series/ce/ce.txt



In [3]:
# Check if file has ben downloaded if not download the file
# This could take a while first time downloading the files (over 5 mins)
# Does this qualify as web scraping? :-)
CESFileName = 'resources/AllCESSeries.txt'
SeriesFileName = 'resources/CESSeries.txt'
IndustryFileName = 'resources/CESIndustries.txt'
if not os.path.exists(CESFileName) :
    url = "https://download.bls.gov/pub/time.series/ce/ce.data.0.AllCESSeries"
    wget.download(url,CESFileName)
    
    url = "https://download.bls.gov/pub/time.series/ce/ce.series"    
    wget.download(url,SeriesFileName)
    
    url = "https://download.bls.gov/pub/time.series/ce/ce.industry"    
    wget.download(url,IndustryFileName)  

allCES =  pd.read_table(CESFileName, sep='\t', header=0,names=['series_id', 'year', 'period', 'value', 'footnote_codes'])    

seriesDF =  pd.read_table(SeriesFileName, sep='\t', header=0,
                          names=['series_id', 'supersector_code','industry_code','data_type_code', 'seasonal', 'series_title', 'footnote_codes','begin_year', 'begin_period', 'end_year', 'end_period'],
                          dtype = {'supersector_code':np.str_,  'industry_code':np.str_,'data_type_code':np.str_})    

industriesDF =  pd.read_table(IndustryFileName, sep='\t', header=0,dtype={'industry_code':np.str_,'naics_code':np.str_})    



In [4]:
# Split Industry from Series ID
allCES['industry_code'] = allCES['series_id'].str.slice(3,11)

seriesDF.drop(['begin_year', 'begin_period', 'end_year', 'end_period'],1,inplace=True)



In [5]:
# Only interested in 2020
# CES Seasonally Adjusted
condition = allCES['year'].isin([2020]) & allCES['series_id'].str.startswith('CES')
CES2020DF = allCES[condition]
CES2020DF.shape

(39251, 6)

In [6]:
# Get month from period and dop period
# month 13 Annual Average      
# https://download.bls.gov/pub/time.series/ce/ce.period
CES2020DF['Month'] = CES2020DF['period'].str.replace('M','').astype(int)
CES2020DF.drop('period',1,inplace=True)

e:\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

e:\Anaconda3\envs\PythonData\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [7]:
# Merge with Industry
CES2020DF = pd.merge(CES2020DF,industriesDF,on='industry_code',how='left')
CES2020DF.shape

(39251, 12)

In [8]:
# Merge with Series DF
CES2020DF = pd.merge(CES2020DF,seriesDF,on='series_id',how='left')
CES2020DF.shape


(39251, 18)

In [9]:
# data_type_code  = 1 all
# data_type_code  = 10  Women employees
# Assume Men =   all - Women

condition = CES2020DF['data_type_code'].isin(['01','10'])
CES2020DF = CES2020DF[condition]

CES2020DF.shape



(4815, 18)

In [10]:
# Browse the data
# uncomment the following lines to browse the data

#d = dtale.show(CES2020DF, ignore_duplicate=True)
#d.open_browser()

In [11]:
CES2020DF.to_pickle("resources/CES2020.pkl",compression='gzip')

In [12]:
# to load Picke file use the following once pickle has been generated
CES2020DF_2=pd.read_pickle('resources/CES2020.pkl',compression='gzip')